In [1]:
# !pip install -r requirements.txt
## Base of notebooks
%load_ext autoreload
%autoreload 2
import tensorflow as tf

In [2]:
from datetime import datetime

now = datetime.now() # current date and time
date = now.strftime("%Y-%m-%d")
img_size = 128
task  = 'binarydensity-pb'

In [3]:
import sys 
sys.path.append('..')

from utils import *
import models

In [4]:
base_data = stack_images(im_size=img_size, directory= "C:\\Users\\baran.tanriverdi\\Final Data")

100%|██████████| 43/43 [00:06<00:00,  6.35it/s]


Al for 5 Cannot found!


100%|██████████| 150/150 [00:17<00:00,  8.72it/s]

Fe2 : 51
Fe5 : 49
Fe10 : 100
Fe20 : 161
Fe200 : 153
Al2 : 43
Al10 : 16
Al20 : 150
Al200 : 136
Pb2 : 53
Pb5 : 50
Pb10 : 97
Pb20 : 150
Pb200 : 150
dict_keys(['Fe2', 'Fe5', 'Fe10', 'Fe20', 'Fe200', 'Al2', 'Al10', 'Al20', 'Al200', 'Pb2', 'Pb5', 'Pb10', 'Pb20', 'Pb200'])


In [5]:
np.asarray(base_data['Fe5']).shape

(49, 128, 128, 3)

In [6]:
# Configyure for current experiment

x1 = []
for imset in [
    base_data['Pb2'], 
    base_data['Pb5'], 
    base_data['Pb10'],
    base_data['Pb20']]:
    for im in imset:
        x1.append(im)
print(np.asarray(x1).shape)

x2 = []
for imset in [base_data['Pb200']]:
    for im in imset:
        x2.append(im)
print(np.asarray(x2).shape)
        

imlist = []
one_hot_vector = np.eye(2)
for x in x1:
    imlist.append([x, one_hot_vector[0]])
for x in x2:
    imlist.append([x, one_hot_vector[1]])  
    
print(len(imlist), len(x1), len(x2))

data = np.asarray(imlist)

np.random.shuffle(data)

x = np.asarray([i[0] / 255.0 for i in data])
y = np.asarray([i[1] for i in data])

len(x), len(y)

boundary = int(len(x) * 0.75)

train_x = x[:boundary]
train_y = y[:boundary]
test_x = x[boundary:]
test_y = y[boundary:]

print(len(train_x), len(test_x))

(350, 128, 128, 3)
(150, 128, 128, 3)
500 350 150
375 125


C:\Users\baran.tanriverdi\AppData\Local\Temp\ipykernel_13600\1692269279.py:29: VisibleDeprecationWarning: Creating an ndarray from ragged nested sequences (which is a list-or-tuple of lists-or-tuples-or ndarrays with different lengths or shapes) is deprecated. If you meant to do this, you must specify 'dtype=object' when creating the ndarray.
  data = np.asarray(imlist)


In [8]:
histories = []
models_ = [
    models.unet(input_shape=(img_size, img_size, 3)), 
    models.vgg5_block(input_shape=(img_size, img_size, 3)),
    #models.vgg16(input_shape=(img_size, img_size, 3)), 
    #models.vgg19(input_shape=(img_size, img_size, 3)),
    #models.resnet50(input_shape=(img_size, img_size, 3)), 
    #models.deeplabv3plus(input_shape=(img_size, img_size, 3)),
    #models.nasnet_mobile(input_shape=(img_size, img_size, 3)), 
    #models.inception_resnetv2(input_shape=(img_size, img_size, 3)), 
    #models.double_unet(input_shape=(img_size, img_size, 3)),
]

for model in tqdm(models_):
    history =model.fit(train_x, train_y, batch_size=10,
                    validation_split=0.2,
                    epochs=25, 
                    #callbacks = tf.keras.callbacks.EarlyStopping(monitor='val_loss',patience=2,),
                    verbose = 1)
    histories.append(history)
    MODEL_NAME = f"{date}_{model.name}_{img_size}_{task}"
    print(MODEL_NAME)
    save_logs(history, MODEL_NAME)
        #save m-odel
    tf.keras.models.save_model(
        model,
        f'./models/{task}/{MODEL_NAME}.h5')
           

C:\Users\baran.tanriverdi\Anaconda3\envs\tf\lib\site-packages\keras\optimizer_v2\adam.py:105: UserWarning: The `lr` argument is deprecated, use `learning_rate` instead.
  super(Adam, self).__init__(name, **kwargs)


Model: "vgg5_block"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 conv2d_17 (Conv2D)          (None, 128, 128, 32)      896       
                                                                 
 conv2d_18 (Conv2D)          (None, 128, 128, 32)      9248      
                                                                 
 max_pooling2d_3 (MaxPooling  (None, 64, 64, 32)       0         
 2D)                                                             
                                                                 
 conv2d_19 (Conv2D)          (None, 64, 64, 64)        18496     
                                                                 
 conv2d_20 (Conv2D)          (None, 64, 64, 64)        36928     
                                                                 
 max_pooling2d_4 (MaxPooling  (None, 32, 32, 64)       0         
 2D)                                                    

  0%|          | 0/2 [00:00<?, ?it/s]

Epoch 1/25
30/30 [==============================] - 1529s 51s/step - loss: 5123.5415 - accuracy: 0.6067 - f1_score: 0.5950 - cohen_kappa: -0.6170 - MatthewsCorrelationCoefficient: -0.0281 - auc: 0.6013 - val_loss: 0.6558 - val_accuracy: 0.7067 - val_f1_score: 0.5852 - val_cohen_kappa: -0.3339 - val_MatthewsCorrelationCoefficient: 0.0000e+00 - val_auc: 0.7429
Epoch 2/25
30/30 [==============================] - 1513s 50s/step - loss: 0.6203 - accuracy: 0.6833 - f1_score: 0.6310 - cohen_kappa: -0.4576 - MatthewsCorrelationCoefficient: 0.0418 - auc: 0.7550 - val_loss: 0.5858 - val_accuracy: 0.7067 - val_f1_score: 0.5852 - val_cohen_kappa: -0.3339 - val_MatthewsCorrelationCoefficient: 0.0000e+00 - val_auc: 0.7787
Epoch 3/25
30/30 [==============================] - 1526s 51s/step - loss: 0.7799 - accuracy: 0.6667 - f1_score: 0.6133 - cohen_kappa: -0.4619 - MatthewsCorrelationCoefficient: -0.0122 - auc: 0.7230 - val_loss: 0.5898 - val_accuracy: 0.7067 - val_f1_score: 0.5852 - val_cohen_kappa:

Epoch 24/25
30/30 [==============================] - 1415s 47s/step - loss: 0.5125 - accuracy: 0.7200 - f1_score: 0.6146 - cohen_kappa: -0.3339 - MatthewsCorrelationCoefficient: 0.0570 - auc: 0.8180 - val_loss: 0.5344 - val_accuracy: 0.7067 - val_f1_score: 0.5852 - val_cohen_kappa: -0.3339 - val_MatthewsCorrelationCoefficient: 0.0000e+00 - val_auc: 0.8012
Epoch 25/25
30/30 [==============================] - 1256s 42s/step - loss: 0.5772 - accuracy: 0.7133 - f1_score: 0.6054 - cohen_kappa: -0.3339 - MatthewsCorrelationCoefficient: -0.0078 - auc: 0.8130 - val_loss: 0.5690 - val_accuracy: 0.6933 - val_f1_score: 0.6001 - val_cohen_kappa: -0.3846 - val_MatthewsCorrelationCoefficient: 0.0179 - val_auc: 0.7474
2022-04-07_unet_128_binarydensity-pb


 50%|█████     | 1/2 [10:25:04<10:25:04, 37504.26s/it]

Epoch 1/25
30/30 [==============================] - 105s 3s/step - loss: 0.6713 - accuracy: 0.6367 - f1_score: 0.6126 - auc: 0.6694 - val_loss: 0.6274 - val_accuracy: 0.7067 - val_f1_score: 0.5852 - val_auc: 0.7792
Epoch 2/25
30/30 [==============================] - 92s 3s/step - loss: 0.6225 - accuracy: 0.6900 - f1_score: 0.6125 - auc: 0.7216 - val_loss: 0.6054 - val_accuracy: 0.7067 - val_f1_score: 0.5852 - val_auc: 0.8231
Epoch 3/25
30/30 [==============================] - 91s 3s/step - loss: 0.5635 - accuracy: 0.7367 - f1_score: 0.6589 - auc: 0.7900 - val_loss: 0.5337 - val_accuracy: 0.7467 - val_f1_score: 0.6697 - val_auc: 0.8347
Epoch 4/25
30/30 [==============================] - 90s 3s/step - loss: 0.5530 - accuracy: 0.7567 - f1_score: 0.7165 - auc: 0.8174 - val_loss: 0.4865 - val_accuracy: 0.7600 - val_f1_score: 0.6943 - val_auc: 0.8801
Epoch 5/25
30/30 [==============================] - 90s 3s/step - loss: 0.4803 - accuracy: 0.7867 - f1_score: 0.7624 - auc: 0.8622 - val_loss: 

100%|██████████| 2/2 [11:02:01<00:00, 19860.83s/it]   
